In [1]:
import pickle
import gzip
import numpy as np
import scipy
import pandas as pd
import keras
import keras.layers as layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from keras.layers import Input, LSTM, Dense, Lambda, Conv1D, Conv2D, AveragePooling2D, AveragePooling1D, Flatten, MaxPooling2D, MaxPooling1D, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.utils import Sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.applications import imagenet_utils
from keras import backend as K
from time import time
from keras import losses
from sklearn.metrics import  roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score, mean_absolute_error
import glob
from PIL import Image
import h5py
import random
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.gaussian_process as gp
import pandas as pd
import pickle
from joblib import dump, load
from sklearn.utils import shuffle
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip install silence_tensorflow

  Created wheel for silence-tensorflow: filename=silence_tensorflow-1.1.1-cp36-none-any.whl size=3743 sha256=15252dd2842f0f6d5e42df374f261dc79fed30113c310ef07472617c7df9378b
  Stored in directory: /root/.cache/pip/wheels/51/0b/35/cf3020764bee61daa81fa249df3a448e3806344a087fc12292
Successfully built silence-tensorflow


In [3]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import tensorflow as tf

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jan  6 01:18:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !rm -R qmc qmc1
    !git clone https://github.com/fagonzalezo/qmc.git
    !mv qmc qmc1
    !mv qmc1/qmc .
else:
    import sys
    sys.path.insert(0, "../")

rm: cannot remove 'qmc': No such file or directory
rm: cannot remove 'qmc1': No such file or directory
Cloning into 'qmc'...
remote: Enumerating objects: 215, done.
remote: Total 215 (delta 0), reused 0 (delta 0), pack-reused 215
Receiving objects: 100% (215/215), 17.06 MiB | 5.65 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [7]:
import qmc.tf.layers as layers
import qmc.tf.models as models

In [8]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((130,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X , y = shuffle(X, y, random_state = 0)
    MAE_opt = 100
    dim_y = 5
    
    beta_opt =  18.343965257225687 
    gamma_opt =  0.027653098129602482 
    learning_rate_opt =  0.0007116317386970352 
    alpha_opt =  0.5050441340504782
    dim_x_opt =  256 
    num_eig_opt =  8

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((64,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(32, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=32, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 0
33/33 [==============================] - 2s 1ms/step
------------MAE = 1.0625
---------------------------------------- 1
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
---------------------------------------- 2
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
---------------------------------------- 3
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
---------------------------------------- 4
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.03125
---------------------------------------- 5
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.015625
---------------------------------------- 6
33/33 [==============================] - 0s 1ms/step
------------MAE = 0.96875
---------------------------------------- 7
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.078125
--------------------

In [9]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((130,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X , y = shuffle(X, y, random_state = 0)
    MAE_opt = 100
    dim_y = 5
    
    beta_opt = 11.168578686120808 
    gamma_opt = 0.019782636050873624 
    learning_rate_opt = 0.0008736519215542466 
    alpha_opt = 0.383147331982576 
    dim_x_opt = 512 
    num_eig_opt = 32

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((64,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(32, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=32, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 0
33/33 [==============================] - 0s 1ms/step
------------MAE = 0.96875
---------------------------------------- 1
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.0
---------------------------------------- 2
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.0
---------------------------------------- 3
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.125
---------------------------------------- 4
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
---------------------------------------- 5
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.078125
---------------------------------------- 6
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.109375
---------------------------------------- 7
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.0625
--------------------------------

In [10]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((130,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X , y = shuffle(X, y, random_state = 0)
    MAE_opt = 100
    dim_y = 5
    
    beta_opt = 14.937963660171226 
    gamma_opt = 0.02192016019576171 
    learning_rate_opt = 0.0003769575135508885 
    alpha_opt = 0.45844374183474923 
    dim_x_opt = 256 
    num_eig_opt = 32

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((64,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(32, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=32, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 0
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.0625
---------------------------------------- 1
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
---------------------------------------- 2
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.015625
---------------------------------------- 3
33/33 [==============================] - 0s 1ms/step
------------MAE = 0.984375
---------------------------------------- 4
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.015625
---------------------------------------- 5
33/33 [==============================] - 0s 992us/step
------------MAE = 1.046875
---------------------------------------- 6
33/33 [==============================] - 0s 1ms/step
------------MAE = 0.953125
---------------------------------------- 7
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.0625
------------------

In [11]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((130,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X , y = shuffle(X, y, random_state = 0)
    MAE_opt = 100
    dim_y = 5
    
    beta_opt = 17.81915647088741 
    gamma_opt = 0.019782636050873624 
    learning_rate_opt = 6.701501049808681e-05 
    alpha_opt = 0.48092427920298797 
    dim_x_opt = 512 
    num_eig_opt = 32

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((64,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(32, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=32, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 0
33/33 [==============================] - 0s 1ms/step
------------MAE = 0.984375
---------------------------------------- 1
33/33 [==============================] - 0s 1ms/step
------------MAE = 0.9375
---------------------------------------- 2
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.03125
---------------------------------------- 3
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
---------------------------------------- 4
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.03125
---------------------------------------- 5
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.078125
---------------------------------------- 6
33/33 [==============================] - 0s 1ms/step
------------MAE = 0.96875
---------------------------------------- 7
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.03125
----------------------

In [12]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((130,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X , y = shuffle(X, y, random_state = 0)
    MAE_opt = 100
    dim_y = 5
    
    beta_opt = 4.526095308003731 
    gamma_opt = 0.026826101784219892 
    learning_rate_opt = 0.0006221125931396382 
    alpha_opt = 0.9805380719114533 
    dim_x_opt = 1024 
    num_eig_opt = 64

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((64,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(32, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=32, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 0
33/33 [==============================] - 0s 2ms/step
------------MAE = 1.03125
---------------------------------------- 1
33/33 [==============================] - 0s 2ms/step
------------MAE = 1.015625
---------------------------------------- 2
33/33 [==============================] - 0s 2ms/step
------------MAE = 1.046875
---------------------------------------- 3
33/33 [==============================] - 0s 2ms/step
------------MAE = 1.09375
---------------------------------------- 4
33/33 [==============================] - 0s 2ms/step
------------MAE = 0.984375
---------------------------------------- 5
33/33 [==============================] - 0s 2ms/step
------------MAE = 1.0625
---------------------------------------- 6
33/33 [==============================] - 0s 2ms/step
------------MAE = 0.921875
---------------------------------------- 7
33/33 [==============================] - 0s 2ms/step
------------MAE = 1.03125
---------------------

In [13]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((130,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X , y = shuffle(X, y, random_state = 0)
    MAE_opt = 100
    dim_y = 5
    
    beta_opt = 7.902870743663927 
    gamma_opt = 0.019782636050873624 
    learning_rate_opt = 0.00017466660484643585 
    alpha_opt = 0.34917795264541507 
    dim_x_opt = 512 
    num_eig_opt = 64

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((64,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(32, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=32, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 0
33/33 [==============================] - 0s 1ms/step
------------MAE = 0.984375
---------------------------------------- 1
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.015625
---------------------------------------- 2
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
---------------------------------------- 3
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.03125
---------------------------------------- 4
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.09375
---------------------------------------- 5
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.078125
---------------------------------------- 6
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.0
---------------------------------------- 7
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
-----------------------

In [14]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/wisconsin/data/wpbc_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((130,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X , y = shuffle(X, y, random_state = 0)
    MAE_opt = 100
    dim_y = 5
    
    beta_opt = 18.357737577708882 
    gamma_opt = 0.09416500056019565 
    learning_rate_opt = 0.00044636852080991385 
    alpha_opt = 0.8888784573520224 
    dim_x_opt = 512 
    num_eig_opt = 32

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((64,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(32, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=32, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 0
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.0
---------------------------------------- 1
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
---------------------------------------- 2
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.078125
---------------------------------------- 3
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.109375
---------------------------------------- 4
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.0625
---------------------------------------- 5
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.109375
---------------------------------------- 6
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.046875
---------------------------------------- 7
33/33 [==============================] - 0s 1ms/step
------------MAE = 1.09375
------------------------

In [ ]:
beta =  2.7976350228789935 
gamma =  0.0007818322758194163 
learning_rate =  0.0009669230439524917 
alpha =  0.778557764997306 
dim_x =  512 
num_eig =  32
